A CNN on 2-columns: both the **global views** and the **faces**

In [3]:
import tensorflow as tf
import time
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

from helpers.data_loading import *
from helpers.CNN_layers import ConvLayer, DenseLayer
from spearman import score_function

In [4]:
y, _ = load_labels()  # scores between 0 and 25 for training instances
y = y.reshape((-1,1))

In [28]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
sess = tf.InteractiveSession(config=config)

input_tf = tf.placeholder(tf.float32, shape=[None, 100, 100, 3])
y_tf = tf.placeholder(tf.float32, shape=[None, 1])

conv1 = ConvLayer(input_tf, 16)
conv1.max_pool()

conv2 = ConvLayer(conv1.output, 32)
conv2.max_pool()

conv3 = ConvLayer(conv2.output, 64)
conv3.max_pool()

fc1 = DenseLayer(conv3.output, 100)
fc1.drop_out()

In [29]:
# second input: global views
input_tf_bis = tf.placeholder(tf.float32, shape=[None, 100, 100, 3])

conv1_bis = ConvLayer(input_tf_bis, 16)
conv1_bis.max_pool()

conv2_bis = ConvLayer(conv1_bis.output, 32)
conv2_bis.max_pool()


fc1_bis = DenseLayer(conv2_bis.output, 100)
fc1_bis.drop_out()

In [30]:
merging_matrix = tf.concat(1, [fc1_bis.output, fc1.output])
merge_layer = DenseLayer(merging_matrix, 100)

In [31]:
fc2 = DenseLayer(merge_layer.output, 1)

# final_output = 24. * (1 / (1 + tf.exp(- fc2.output)))  # logit to normalize
final_output = fc2.output

In [32]:
mse = (
    tf.reduce_mean((final_output - y_tf)**2) + 1*(
        tf.nn.l2_loss(conv1.W) +
        tf.nn.l2_loss(conv2.W) +
        tf.nn.l2_loss(conv3.W) +
        tf.nn.l2_loss(conv1_bis.W) +
        tf.nn.l2_loss(conv2_bis.W) +
        tf.nn.l2_loss(fc1.W) +
        tf.nn.l2_loss(fc2.W) + 
        tf.nn.l2_loss(fc1_bis.W)+
        tf.nn.l2_loss(merge_layer.W)
    )
)
learning_rate = tf.placeholder(tf.float32, shape=[])
#  train_step = tf.train.AdamOptimizer(learning_rate).minimize(mse)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(mse)

In [33]:
sess.run(tf.global_variables_initializer())

In [34]:
mse.eval(feed_dict={
    input_tf: faces_batch(np.arange(1, 51), black_white=False)[0], 
    input_tf_bis: global_views_batch(np.arange(1, 51), black_white=False)[0],
    y_tf: y[0:50,:], 
    fc1.keep_prob: 1.0,
    fc1_bis.keep_prob: 1.0
})

13237.32

In [35]:
type(local_views_batch(np.arange(1, 11))[0][0,0,0])

numpy.float32

In [36]:
# WARNING indices for y and when calling global_views_batch differ from 1
# we create a quick batch generator for the training and the testing

# sample train/test indices # WARNING indexed from 1!!!
train = np.arange(1, 9501)
test = np.arange(9501, 10001)

def train_batch(batch_size=50, black_white=True):
    nb_batch = len(train) // batch_size
    while 1:
        for i in range(nb_batch):
            in_batch = train[i*batch_size:((i+1)*batch_size)]
            X_batch, _ = faces_batch(in_batch, black_white=black_white)
            X_batch_bis, _ = global_views_batch(in_batch, black_white=black_white)
            y_batch = y[in_batch-1]  # WARNING Y is indexed from 0!!!
            y_batch = y_batch.astype(np.float32)
            yield X_batch/255. -.5 , y_batch, X_batch_bis/255. -.5 
            
def test_batch(batch_size=50, black_white=True):
    nb_batch = len(test) // batch_size
    while 1:
        for i in range(nb_batch):
            in_batch = test[i*batch_size:((i+1)*batch_size)]
            X_batch, _ = faces_batch(in_batch, black_white=black_white)
            X_batch_bis, _ = global_views_batch(in_batch, black_white=black_white)
            y_batch = y[in_batch-1]  # WARNING Y is indexed from 0!!!
            y_batch = y_batch.astype(np.float32)
            yield X_batch/255. -.5 , y_batch, X_batch_bis/255. -.5 

In [37]:
train_gen = train_batch(100, black_white=False)
test_gen = test_batch(300, black_white=False)

In [38]:
X_test, y_test, X_test_bis = next(test_gen)
#X_test = X_test.reshape((-1, 200, 200, 1))
print X_test.shape
print X_test_bis.shape
print y_test.shape

(300, 100, 100, 3)
(300, 100, 100, 3)
(300, 1)


In [39]:
def epoch(lr=1e-4):
    X_train, y_train, X_train_bis = next(train_gen)
    #X_train = X_train.reshape((-1, 200, 200, 1))
    # X_test, y_test = next(test_gen)
    # X_test = X_test.reshape((-1, 200, 200, 1))
    train_step.run(feed_dict={
        input_tf: X_train, 
        input_tf_bis: X_train_bis,
        y_tf: y_train,
        fc1.keep_prob: 1,
        fc1_bis.keep_prob: 1, 
        learning_rate: lr
    })
    train_mse = sess.run(mse, feed_dict={input_tf:X_train, input_tf_bis: X_train_bis,
                                         y_tf: y_train, fc1.keep_prob: 1.0, fc1_bis.keep_prob: 1.0})
    
    # preds on test
    [test_mse, preds] = sess.run([mse, final_output], feed_dict={input_tf: X_test, input_tf_bis: X_test_bis,
                                         fc1.keep_prob: 1.0, fc1_bis.keep_prob: 1.0, y_tf: y_test})

    current_spearman = score_function(y_pred=np.round(preds), y_true=y_test)
    
    return {"train_mse":train_mse, "test_mse":test_mse, "test_spearman":current_spearman, 'lr': lr}

In [40]:
learning_history = {'iterations':[], 'train_mse':[], 'test_mse':[], 'test_spearman':[], 'lr':[]}

def update_history(learning_history, epoch):
    learning_history['iterations'].append(len(learning_history['iterations']))
    for name in ['train_mse', 'test_mse', 'test_spearman', 'lr']:
        learning_history[name].append(epoch[name])

In [41]:
sess.run(tf.global_variables_initializer())

In [42]:
window = 3000  # plot only last 500 points
for lr in [10**-4, 5*10**-5, 10**-5, 10**-6]:
    for i in range(500):  # plus 2k before at 10**-4
        update_history(learning_history, epoch(lr=lr))
        if (i+1) % 10 == 0:
            clear_output()
            print "i=%i"%i
            f, axarr = plt.subplots(1, 2, figsize=(13, 3))
            axarr[0].plot(learning_history['iterations'][:-window:-1][::-1], learning_history['train_mse'][:-window:-1][::-1], label="MSE (train)")
            axarr[0].plot(learning_history['iterations'][:-window:-1][::-1], learning_history['test_mse'][:-window:-1][::-1], label="MSE (test)")
            axarr[1].plot(learning_history['iterations'][:-window:-1][::-1], learning_history['test_spearman'][:-window:-1][::-1], label="Spearman (test)")
            axarr[0].legend()
            axarr[1].set_title("Spearman on test set")
            axarr[1].grid()
            plt.show()

i=209


KeyboardInterrupt: 

In [137]:
final_output.eval(feed_dict={input_tf: X_test, input_tf_bis: X_test_bis,
                                         fc1.keep_prob: 1.0, fc1_bis.keep_prob: 1.0})

array([[ 14.07024288],
       [ 14.73140335],
       [ 10.28329563],
       [ 14.07658672],
       [ 14.65074444],
       [ 12.20059299],
       [  7.39949369],
       [ 13.57999039],
       [ 12.09169292],
       [ 10.88322067],
       [ 14.57351017],
       [ 14.22776508],
       [ 12.37867165],
       [ 15.71650124],
       [ 11.896842  ],
       [ 16.28238869],
       [ 10.1132946 ],
       [ 13.42016602],
       [ 14.19437027],
       [ 10.3393259 ],
       [ 10.31546307],
       [ 12.40370083],
       [ 12.96519852],
       [ 12.44005966],
       [  9.39571095],
       [ 12.69921875],
       [ 11.12191963],
       [ 11.23336029],
       [ 14.4354744 ],
       [ 15.15467739],
       [ 13.87317371],
       [ 11.04125404],
       [  9.13256836],
       [ 11.70853806],
       [ 13.67470264],
       [ 12.72490597],
       [ 14.18181992],
       [ 14.65247059],
       [ 11.56665421],
       [ 15.85756302],
       [ 13.97681236],
       [ 14.07945156],
       [ 11.41934204],
       [  9